In [1]:
from pathlib import Path

import pandas as pd
from chess.engine import SimpleEngine

from metrics.themes import legal, get_unique_puzzle_from_fen, counter_intuitive

In [ ]:
df = pd.read_csv("./processed_puzzles.csv")
print(df["legal"].mean(), df["unique"].mean(), df["counter_intuitive"].mean(), (df["unique"] & df["counter_intuitive"]).mean())
# Lichess puzzle dataset (sample of 1000) equivalents (100%, 81.7%, 4.3%, 4.1%)
df.head()

0.968 0.09670563230605739 0.23698193411264612 0.001


,index,FEN,rating,theme,legal,unique,counter_intuitive,themes
0,1,8/p7/1p5p/6p1/1P3kP1/P6P/2N3K1/2b5 w - - 3 45,3246,mateIn3,True,False,False,NaN
1,2,3R4/p4r1k/7p/p7/3b2p1/6P1/6PP/1qQ2R1K b - - 3 35,475,quietMove,True,True,False,endgame mateIn2 mate xRayAttack fork short
2,3,2r3k1/5ppp/1p1b4/1p6/8/P2PR2P/1R4P1/2B4K b - -...,571,mate,True,True,False,endgame mateIn2 mate hangingPiece short
3,4,2k3r1/ppp2p2/2p1q3/2b4p/4P3/2NB1NPb/PPP1Q3/R3R...,951,mateIn2,True,False,True,NaN
4,5,8/6pp/5p2/p7/1p2kP1P/1PP3P1/1P2K3/8 b - - 0 44,1821,zugzwang,True,False,True,NaN


In [15]:
base_path = Path(".")
dataset_path = base_path / "dataset" / "dataset.csv"
generation_path = base_path / "generated_fens.txt"
stockfish_path = base_path / ".." / "Stockfish" / "src" / "stockfish"

In [ ]:
generations = pd.read_csv(generation_path, header=None, names=["index", "FEN", "rating", "theme", "legal"])
generated_fens = set(generations["FEN"])

overlap = 0
for chunk in pd.read_csv(dataset_path, chunksize=100_000):
    overlap += chunk["Puzzle_FEN"].isin(generated_fens).sum()

print(overlap)

0


In [ ]:
n = 1000
n_unique_lichess = 0
n_counter_intuitive_lichess = 0
n_puzzles_lichess = 0
with SimpleEngine.popen_uci(stockfish_path) as engine:
    for chunk in pd.read_csv(dataset_path, chunksize=n):
        for i, fen in enumerate(chunk["Puzzle_FEN"]):
            unique = get_unique_puzzle_from_fen(fen, engine) is not None
            counter_intuitive_position = counter_intuitive(fen, engine)
            n_unique_lichess += unique
            n_counter_intuitive_lichess += counter_intuitive_position
            n_puzzles_lichess += unique and counter_intuitive_position
            if i % 10 == 0: print(i, fen)
        break

print(n_unique_lichess / n, n_counter_intuitive_lichess / n, n_puzzles_lichess / n)


0 r6k/pp2r2p/4Rp1Q/3p4/8/1N1P2b1/PqP3PP/7K w - - 0 25


KeyboardInterrupt: 